# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 8:30AM,241241,10,0085840060,ISDIN Corporation,Released
1,Jul 7 2022 8:30AM,241242,10,0085840580,ISDIN Corporation,Released
2,Jul 7 2022 8:30AM,241242,10,0085840581,ISDIN Corporation,Released
3,Jul 7 2022 8:30AM,241242,10,0085840576,ISDIN Corporation,Released
4,Jul 7 2022 8:30AM,241242,10,0085840577,ISDIN Corporation,Released
5,Jul 7 2022 8:30AM,241242,10,0085840585,ISDIN Corporation,Released
6,Jul 7 2022 8:30AM,241242,10,0085840584,ISDIN Corporation,Released
7,Jul 7 2022 8:30AM,241242,10,0085840583,ISDIN Corporation,Released
8,Jul 7 2022 8:30AM,241242,10,0085840582,ISDIN Corporation,Released
9,Jul 7 2022 8:30AM,241242,10,0085840579,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,241258,Released,6
53,241259,Released,4
54,241260,Released,14
55,241261,Released,2
56,241263,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241258,NaN,NaN,6.0
241259,NaN,NaN,4.0
241260,NaN,NaN,14.0
241261,NaN,NaN,2.0
241263,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241147,0.0,0.0,3.0
241161,0.0,0.0,44.0
241163,0.0,0.0,58.0
241164,0.0,0.0,8.0
241169,2.0,1.0,29.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241147,0,0,3
241161,0,0,44
241163,0,0,58
241164,0,0,8
241169,2,1,29


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241147,0,0,3
1,241161,0,0,44
2,241163,0,0,58
3,241164,0,0,8
4,241169,2,1,29


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241147,,,3
1,241161,,,44
2,241163,,,58
3,241164,,,8
4,241169,2,1,29


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation
1,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation
11,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation
48,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc."
49,Jul 7 2022 8:30AM,241247,10,"Goodwin Biotechnology, Inc."
50,Jul 7 2022 8:30AM,241250,10,Eywa Pharma Inc.
51,Jul 7 2022 8:29AM,241263,10,"Methapharm, Inc."
56,Jul 7 2022 8:17AM,241261,10,"Amcyte Pharma, Inc."
58,Jul 7 2022 8:15AM,241260,10,ISDIN Corporation
72,Jul 7 2022 8:09AM,241259,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,,,11
1,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation,,,32
2,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation,,,5
3,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc.",,,1
4,Jul 7 2022 8:30AM,241247,10,"Goodwin Biotechnology, Inc.",,,1
5,Jul 7 2022 8:30AM,241250,10,Eywa Pharma Inc.,,,1
6,Jul 7 2022 8:29AM,241263,10,"Methapharm, Inc.",,,5
7,Jul 7 2022 8:17AM,241261,10,"Amcyte Pharma, Inc.",,,2
8,Jul 7 2022 8:15AM,241260,10,ISDIN Corporation,,,14
9,Jul 7 2022 8:09AM,241259,10,Emerginnova,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,11,,
1,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation,32,,
2,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation,5,,
3,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc.",1,,
4,Jul 7 2022 8:30AM,241247,10,"Goodwin Biotechnology, Inc.",1,,
5,Jul 7 2022 8:30AM,241250,10,Eywa Pharma Inc.,1,,
6,Jul 7 2022 8:29AM,241263,10,"Methapharm, Inc.",5,,
7,Jul 7 2022 8:17AM,241261,10,"Amcyte Pharma, Inc.",2,,
8,Jul 7 2022 8:15AM,241260,10,ISDIN Corporation,14,,
9,Jul 7 2022 8:09AM,241259,10,Emerginnova,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,11,,
1,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation,32,,
2,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation,5,,
3,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc.",1,,
4,Jul 7 2022 8:30AM,241247,10,"Goodwin Biotechnology, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,11.0,NaN,NaN
1,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation,32.0,NaN,NaN
2,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation,5.0,NaN,NaN
3,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jul 7 2022 8:30AM,241247,10,"Goodwin Biotechnology, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,11.0,0.0,0.0
1,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation,32.0,0.0,0.0
2,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation,5.0,0.0,0.0
3,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jul 7 2022 8:30AM,241247,10,"Goodwin Biotechnology, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7477537,493.0,0.0,0.0
12,482386,2.0,2.0,0.0
16,964966,3.0,2.0,0.0
19,964919,67.0,0.0,8.0
20,723587,58.0,1.0,2.0
21,1929920,7.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7477537,493.0,0.0,0.0
1,12,482386,2.0,2.0,0.0
2,16,964966,3.0,2.0,0.0
3,19,964919,67.0,0.0,8.0
4,20,723587,58.0,1.0,2.0
5,21,1929920,7.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,493.0,0.0,0.0
1,12,2.0,2.0,0.0
2,16,3.0,2.0,0.0
3,19,67.0,0.0,8.0
4,20,58.0,1.0,2.0
5,21,7.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,493.0
1,12,Released,2.0
2,16,Released,3.0
3,19,Released,67.0
4,20,Released,58.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,8.0,2.0,0.0
Executing,0.0,2.0,2.0,0.0,1.0,1.0
Released,493.0,2.0,3.0,67.0,58.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,8.0,2.0,0.0
1,Executing,0.0,2.0,2.0,0.0,1.0,1.0
2,Released,493.0,2.0,3.0,67.0,58.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,8.0,2.0,0.0
1,Executing,0.0,2.0,2.0,0.0,1.0,1.0
2,Released,493.0,2.0,3.0,67.0,58.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()